In [2]:
!pip install torch_scatter torcheeg torch_geometric -qq 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 7.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 64.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB

In [3]:
#!rm -rf tmp_out

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset, WeightedRandomSampler
from torcheeg.datasets import SEEDIVDataset
from torcheeg import transforms
import scipy.signal as signal
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [5]:
# Set device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Set a fixed random seed for reproducibility across different libraries.
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
set_seed(42)

In [7]:
def BandPassFilter(eeg_data):
    b, a = signal.butter(4, Wn=[1.0, 75.0], btype='bandpass', fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [8]:
def Notch(eeg_data):
    b, a = signal.iirnotch(w0=50.0, Q=30.0, fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [9]:
# 2. Define Preprocessing
t_transform = transforms.Compose([
    transforms.Lambda(BandPassFilter),
    transforms.Lambda(Notch),
    transforms.BaselineRemoval(),
    transforms.MeanStdNormalize(),
    transforms.To2d()
    
])

In [10]:
dataset = SEEDIVDataset(
    io_path='./tmp_out/seed_iv',
    root_path='/kaggle/input/seed-iv/eeg_raw_data',
    offline_transform=t_transform,
    label_transform=transforms.Select('emotion'),
    chunk_size=800,  # 4 seconds
    num_worker=0
)

[2025-12-05 14:44:06] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv.
[2025-12-05 14:44:06] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 1it [00:02,  2.47s/it]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 11it [00:02,  5.84it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 21it [00:02, 12.64it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 32it [00:02, 21.56it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 42it [00:02, 30.67it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 53it [00:02, 41.60it/s]
[RECORD /kaggle/input/

In [11]:
BATCH_SIZE = 64         
MAX_EPOCHS = 100
PATIENCE = 15                
SCHED_FACTOR = 0.3
SCHED_PATIENCE = 5
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 5e-4
CLIP_GRAD = 1.0  

In [12]:
df = dataset.info
counts = df['emotion'].value_counts().sort_index()
print("Class Counts:\n", counts)

Class Counts:
 emotion
0    10170
1    10245
2     9225
3     7935
Name: count, dtype: int64


In [15]:
# Split by Trial ID
# SEED-IV has 24 trials (videos) per session.
# 80% of VIDEOS for training (19 videos), 20% for testing (5 videos).
all_trial_ids = list(range(1, 25))

random.seed(42)
test_trial_ids = random.sample(all_trial_ids, 5)
train_trial_ids = [t for t in all_trial_ids if t not in test_trial_ids]

train_indices = df[df['trial_id'].isin(train_trial_ids)].index.tolist()
test_indices = df[df['trial_id'].isin(test_trial_ids)].index.tolist()

# Create Subsets & Loaders
train_set = Subset(dataset, train_indices)
test_set = Subset(dataset, test_indices)

#train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [16]:
labels = df["emotion"].values

train_labels = labels[train_indices]

class_counts = np.bincount(train_labels)
class_weights = 1.0 / class_counts
sample_weights = class_weights[train_labels]

sampler = WeightedRandomSampler(
    weights=torch.DoubleTensor(sample_weights),
    num_samples=len(sample_weights),
    replacement=True
)

In [17]:
train_loader = DataLoader(
    Subset(dataset, train_indices),
    batch_size=BATCH_SIZE,
    sampler=sampler,
    num_workers=0,
    pin_memory=(device.type == "cuda")
)


In [18]:
test_loader = DataLoader(
    Subset(dataset, test_indices),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=(device.type == "cuda")
)

In [19]:
class EEG_CNN(nn.Module):
    
    def __init__(self, in_channels=1, num_classes=4):
        super().__init__()

        self.print_dims = True

        # ----- Block 1: Temporal Conv (Preserve Spatial, Reduce Temporal) -----
        # 62x800 -> 62x(800 - 30 + 30) = 62x800
        self.temporal_conv = nn.Sequential(
            nn.ZeroPad2d((15, 15, 0, 0)), 
            nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(1, 31), stride=(1, 1), padding=0),
            nn.BatchNorm2d(20, affine=False),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4) 
        )

        # ----- Block 2: Spatial Conv (Reduce Spatial, Reduce Temporal slightly) -----
        # Input: [B, 20, 62, 800]
        # kernel_size=(62, 1) 
        
        self.spatial_conv = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=40, kernel_size=(62, 1), stride=(1, 1), padding=0), 
            nn.LeakyReLU(),
            nn.Dropout(p=0.4)
        )

        # ----- Block 3: Temporal Downsampling (Pooling) -----
        # Input: [B, 40, 1, 800]
        # MaxPool2d(kernel=(1, 4), stride=(1, 4))
        self.temporal_pool1 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4)), # 800 -> 200
            nn.BatchNorm2d(40, affine=False),
            nn.LeakyReLU()
        )

        # ----- Block 4: Deep Temporal Feature Extraction -----
        # Input: [B, 40, 1, 200]
        self.deep_conv = nn.Sequential(
            nn.Conv2d(in_channels=40, out_channels=80, kernel_size=(1, 11), stride=(1, 1), padding=(0, 5)), # 200
            nn.BatchNorm2d(80, affine=False),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4), 
            
            nn.Conv2d(in_channels=80, out_channels=160, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3)), # 200
            nn.BatchNorm2d(160, affine=False),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4)
        )
        
        # ----- Global Pooling & Classification -----
        # Input: [B, 160, 1, 200]
        self.gap = nn.AdaptiveAvgPool2d((1, 1)) # [B, 160, 1, 1]

        self.classifier = nn.Sequential(
            nn.Flatten(), # [B, 160]
            nn.Linear(160, num_classes)
        )

    def forward(self, x):
        
        if self.print_dims: print("Input:", x.shape) # [B, 1, 62, 800]

        # Block 1
        x = self.temporal_conv(x)
        if self.print_dims: print("After Temporal Conv (B1):", x.shape) # [B, 20, 62, 800]

        # Block 2
        x = self.spatial_conv(x)
        if self.print_dims: print("After Spatial Conv (B2):", x.shape) # [B, 40, 1, 800]

        # Block 3
        x = self.temporal_pool1(x)
        if self.print_dims: print("After Temporal Pool (B3):", x.shape) # [B, 40, 1, 200]
            
        # Block 4
        x = self.deep_conv(x)
        if self.print_dims: print("After Deep Conv (B4):", x.shape) # [B, 160, 1, 200]

        # Global Pooling
        x = self.gap(x) 
        if self.print_dims: print("After Global Avg Pool (GAP):", x.shape) # [B, 160, 1, 1]
        
        if self.print_dims:
            self.print_dims = False
        
        x = self.classifier(x)
        return x

In [20]:
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))


def train_epoch(model, loader, criterion, optimizer, clip_grad=None):
    model.train()
    loss_sum = 0.0
    correct = 0
    total = 0

    for X, y in loader:
        X = X.to(device).float()
        y = y.to(device).long()

        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            out = model(X)
            loss = criterion(out, y)

        scaler.scale(loss).backward()

        if clip_grad:
            # unscale then clip
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()

        batch = y.size(0)
        loss_sum += loss.item() * batch
        correct += (out.argmax(1) == y).sum().item()
        total += batch

    return loss_sum / total, 100.0 * correct / total


/tmp/ipykernel_81/4178704967.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))


In [21]:
def evaluate(model, loader, criterion):
    model.eval()
    loss_sum = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device).float()
            y = y.to(device).long()
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                out = model(X)
                loss = criterion(out, y)

            batch = y.size(0)
            loss_sum += loss.item() * batch
            correct += (out.argmax(1) == y).sum().item()
            total += batch

    return loss_sum / total, 100.0 * correct / total

In [22]:
def train_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, scheduler,
                              max_epochs=MAX_EPOCHS, patience=PATIENCE, clip_grad=CLIP_GRAD):
    best_acc = 0.0
    best_state = None
    counter = 0

    for epoch in range(max_epochs):
        # free small fragments each epoch 
        if device.type == 'cuda':
            torch.cuda.empty_cache()
            
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, clip_grad=clip_grad)
        val_loss, val_acc = evaluate(model, val_loader, criterion)

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1:02d} | Train={train_acc:.2f}% | Val={val_acc:.2f}% | ValLoss={val_loss:.4f}")

        if val_acc > best_acc + 1e-4:   
            best_acc = val_acc
            best_state = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping (no improvement for {patience} epochs).")
                break

    return best_acc, best_state


In [23]:
# ------------------ Prepare model, optimizer, scheduler ------------------
from torch.optim.lr_scheduler import ReduceLROnPlateau
model = EEG_CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=SCHED_FACTOR, patience=SCHED_PATIENCE, verbose=True)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [25]:
# ------------------ Run training ------------------
best_acc, best_state = train_with_early_stopping(
    model, train_loader, test_loader, criterion, optimizer, scheduler,
    max_epochs=MAX_EPOCHS, patience=PATIENCE, clip_grad=CLIP_GRAD
)

if best_state is not None:
    model.load_state_dict(best_state)

test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\nFinal Test Accuracy: {test_acc:.2f}%  (best seen during training: {best_acc:.2f}%)")


/tmp/ipykernel_81/4178704967.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Input: torch.Size([64, 1, 62, 800])
After Temporal Conv (B1): torch.Size([64, 20, 62, 800])
After Spatial Conv (B2): torch.Size([64, 40, 1, 800])
After Temporal Pool (B3): torch.Size([64, 40, 1, 200])
After Deep Conv (B4): torch.Size([64, 160, 1, 200])
After Global Avg Pool (GAP): torch.Size([64, 160, 1, 1])


/tmp/ipykernel_81/1341851955.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Epoch 01 | Train=47.78% | Val=44.66% | ValLoss=1.2667
Epoch 02 | Train=58.01% | Val=47.34% | ValLoss=1.2137
Epoch 03 | Train=62.44% | Val=49.61% | ValLoss=1.2396
Epoch 04 | Train=64.20% | Val=40.49% | ValLoss=1.4741
Epoch 05 | Train=65.58% | Val=53.37% | ValLoss=1.1571
Epoch 06 | Train=66.54% | Val=46.51% | ValLoss=1.3653
Epoch 07 | Train=67.91% | Val=51.24% | ValLoss=1.2138
Epoch 08 | Train=68.40% | Val=43.80% | ValLoss=1.4297
Epoch 09 | Train=68.70% | Val=45.32% | ValLoss=1.4631
Epoch 10 | Train=70.19% | Val=52.17% | ValLoss=1.2869
Epoch 11 | Train=69.75% | Val=50.58% | ValLoss=1.2950
Epoch 12 | Train=72.71% | Val=46.15% | ValLoss=1.3904
Epoch 13 | Train=73.53% | Val=54.28% | ValLoss=1.2070
Epoch 14 | Train=74.15% | Val=52.30% | ValLoss=1.3943
Epoch 15 | Train=74.55% | Val=50.42% | ValLoss=1.2844
Epoch 16 | Train=74.37% | Val=49.08% | ValLoss=1.3586
Epoch 17 | Train=74.57% | Val=43.69% | ValLoss=1.5455
Epoch 18 | Train=75.35% | Val=52.88% | ValLoss=1.2485
Epoch 19 | Train=75.85% | Va